In [6]:
!pip install --quiet pathway bokeh


In [7]:
from google.colab import files, drive
uploaded = files.upload()
fname = next(iter(uploaded))


Saving dataset.csv to dataset (2).csv


In [8]:
import pandas as pd, numpy as np
df = pd.read_csv(fname)
df.columns = df.columns.str.strip().str.lower()
df['timestamp'] = pd.to_datetime(df['lastupdateddate'] + ' ' + df['lastupdatedtime'],
                                 dayfirst = True, errors = 'coerce')
df = df.dropna(subset = ['timestamp']).sort_values(['timestamp','systemcodenumber']).reset_index(drop = True)


In [9]:
import pandas as pd, numpy as np
df = pd.read_csv(fname)
df.columns = df.columns.str.strip().str.lower()
df['timestamp'] = pd.to_datetime(df['lastupdateddate'] + ' ' + df['lastupdatedtime'],
                                 dayfirst = True, errors = 'coerce')
df = df.dropna(subset = ['timestamp']).sort_values(['timestamp','systemcodenumber']).reset_index(drop = True)


In [10]:
traffic_map = {'low':0,'medium':1,'high':2}
veh_map = {'bike':0.7,'car':1.0,'truck':1.5}
df['traffic'] = df['trafficconditionnearby'].map(traffic_map).fillna(1)
df['veh_weight'] = df['vehicletype'].map(veh_map).fillna(1)
lots = df[['systemcodenumber','latitude','longitude']].drop_duplicates()
import math
def hv(lat1,lon1,lat2,lon2):
    r = 6371
    p = math.pi/180
    dlat = (lat2-lat1)*p
    dlon = (lon2-lon1)*p
    a = math.sin(dlat/2)**2 + math.cos(lat1*p)*math.cos(lat2*p)*math.sin(dlon/2)**2
    return 2*r*math.asin(math.sqrt(a))
coords = lots.set_index('systemcodenumber').T.to_dict()
neigh = {}
ids = list(coords)
for i in ids:
    lat1,lon1 = coords[i]['latitude'],coords[i]['longitude']
    dist = sorted([(j,hv(lat1,lon1,coords[j]['latitude'],coords[j]['longitude'])) for j in ids if j!=i], key = lambda x:x[1])
    neigh[i] = [k for k,d in dist[:3]]


In [11]:
base = 10.0
def baseline(prev,occ,cap):
    return np.clip(prev + 0.05*(occ/cap), 5, 20)
def demand(row):
    x = 0.5*(row.occupancy/row.capacity) + 0.2*row.queuelength - 0.2*row.traffic + 0.3*row.isspecialday + 0.3*row.veh_weight
    return x
d_min,d_max = df.apply(demand,axis=1).agg(['min','max'])
def dynamic(prev,row,comp):
    n = (demand(row)-d_min)/(d_max-d_min+1e-9)
    cand = np.clip(base*(1+1.0*n),5,20)
    if row.occupancy>=0.95*row.capacity and comp< prev: cand = max(cand-1,5)
    if row.occupancy<=0.4*row.capacity and comp> prev: cand = min(cand+1,20)
    return 0.7*prev + 0.3*cand


In [12]:
from collections import defaultdict
history = defaultdict(list)
prices = {k:base for k in coords}
record = []
for r in df.itertuples():
    lot = r.systemcodenumber
    comp_mean = np.mean([prices[n] for n in neigh[lot]])
    p = dynamic(prices[lot],r,comp_mean)
    prices[lot] = p
    history[lot].append((r.timestamp,p))
    record.append((r.timestamp,lot,p))
price_df = pd.DataFrame(record,columns=['ts','lot','price'])


In [13]:
from bokeh.plotting import output_notebook, figure, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
output_notebook()
src = {lot:ColumnDataSource(dict(t=[x for x,_ in history[lot]],
                                 y=[y for _,y in history[lot]])) for lot in history}
plots=[]
for lot,s in src.items():
    p = figure(title=lot,x_axis_type='datetime',height=250,width=400)
    p.line('t','y',source=s)
    plots.append(p)
show(gridplot(plots,ncols=2))
